# Assignment&nbsp;7:&nbsp;Neural&nbsp;Networks&nbsp;using&nbsp;Keras&nbsp;and&nbsp;Tensorflow&nbsp;


Please see the associated document for questions. If you have problems with Keras and Tensorflow on your local installation please make sure they are updated. On Google Colab this notebook runs.

In [1]:
# imports
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
# Hyper-parameters data-loading and formatting

batch_size = 128
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28

(x_train, lbl_train), (x_test, lbl_test) = mnist.load_data()

if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

11493376/11490434 [==============================] - 0s 0us/step


## Preprocessing

In order to simplify vector operations we normalize the pixels by scaling them from the range [0,255] down to [0,1], dividing each pixel by its max value 255. However, we first need to cast the elements of the tensors to `float` as python's division assignment operator `/=`, doesn't work with `integers` when the ouput is `float`

In [3]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train /= 255
x_test /= 255

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)


x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


Next, we create a one-hot encoding of the digits 0-9, as classification models generally output a one-hot encoded vector with the same number of dimension as classes (10) in the dataset.

In [4]:
y_train = keras.utils.to_categorical(lbl_train, num_classes)
y_test = keras.utils.to_categorical(lbl_test, num_classes)

print('y_train shape, one-hot encoded:', y_train.shape)
print('y_test shape, one-hot encoded:', y_test.shape)  

y_train shape, one-hot encoded: (60000, 10)
y_test shape, one-hot encoded: (10000, 10)


### Generating and training a model
At this point, the dataset has been prepared and it's time to generate, tweak and fit a model for prediction. Here, a sequential model is used, appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor..

In [5]:
## Define model ##
model = Sequential()

#### Layers used in the model
The first layer is added to flatten the input such that the number of dimensions correspond to the number of elements in the input tensor, since the next layer expects a vector as input. More specifically, each image is converted from a 28 x 28 matrix to a vector with 784 elements, 
Another reason for this is to couple information that exists vertically as well as horizontally. There is no way to *couple* the information *across rows* in the pixel arrays representing the images, without either flatting or using a 2-dimensional kernel like a convolution (or maybe some other operation we haven’t considered yet). Without doing this, you can infer information on a row-by-row basis (“horizontally”), but there’s no way to combine information that exists across rows -- it’s like you have a set of disconnected horizontal slices of an image, but no way to “see” the vertical structures, until you either flatten or switch to 2d convolution. 

In [6]:
model.add(Flatten())

The next two fully connected layers are what gives the network the complexity to, when correctly fitted, recognize the handwritten digits. These layers specify two arguments, the first one being the output format or the number neurons and the second one specfiying the **Rectified Linear Unit** as the activation function for the layers neurons. This activation function is what allows the layer to model nonlinear functions.

In [7]:
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))

The final, so called, output layer should have it's output shape set to the expected output of the model. For the handwritten digit scenario, this might be to use one neuron per digit, in other words, a vector of 10 nodes. 
this layer specify a different activation function,  Softmax, which outputs a vector that represents the probability distributions of a list of potential outcomes (digits 0-9) allowing the neural network to output the digit with highest probability value.

In [8]:
model.add(Dense(num_classes, activation='softmax'))

#### Model configuration
Below, the model is configured to use a categorical cross-entropy loss function, suitable for classification tasks like this one. Compared to other common loss functions, such as the squared error cost function, catergorical cross-entropy does not suffer from the same learning slow downs as some of the others do. The optimizer is then set to use mini-batched gradient descent with a learning rate of 0.1. Lastly, the accuracy is specified as the evaluation metric used when training and testing the model.

In [9]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.SGD(lr=0.1),
    metrics=["accuracy"],
)

As we can se from the summary below, our final model contains 4 layers that 

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


At this point the model is ready for training. Here, two hyperparameters `batch_size` and `epochs` can be used to tweak the training process. The prior refers to the number of training samples that are passed through the network at a time, while the latter refers to the number of times the neural network is exposed to the entire dataset. 

Tweaking these hyperparameters might both improve and worsen a model's performance. For instance, gradient descent typically doesn't reach a local or global minima after just one epoch. Hence, increasing the number of epochs might improve results up to a point were the model run the risk of overfitting, worsening it's performance on new data.

In [10]:
fit_info = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test),
)

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: {}, Test accuracy {}".format(loss, accuracy))

Epoch 1/10
469/469 [==============================] - 4s 7ms/step - loss: 0.7864 - accuracy: 0.7741 - val_loss: 0.2596 - val_accuracy: 0.9237
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2494 - accuracy: 0.9274 - val_loss: 0.1964 - val_accuracy: 0.9415
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1859 - accuracy: 0.9454 - val_loss: 0.1588 - val_accuracy: 0.9527
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1555 - accuracy: 0.9542 - val_loss: 0.1467 - val_accuracy: 0.9568
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1300 - accuracy: 0.9616 - val_loss: 0.1256 - val_accuracy: 0.9620
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1132 - accuracy: 0.9673 - val_loss: 0.1185 - val_accuracy: 0.9638
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0990 - accuracy: 0.9706 - val_loss: 0.1065 - val_accuracy: 0.9672
Epoch 

### Question 4) Auto-Encoder for denoising


In [12]:
import numpy as np
def salt_and_pepper(input, noise_level=0.5):
    """
    This applies salt and pepper noise to the input tensor - randomly setting bits to 1 or 0.
    Parameters
    ----------
    input : tensor
        The tensor to apply salt and pepper noise to.
    noise_level : float
        The amount of salt and pepper noise to add.
    Returns
    -------
    tensor
        Tensor with salt and pepper noise applied.
    """
    # salt and pepper noise
    a = np.random.binomial(size=input.shape, n=1, p=(1 - noise_level))
    b = np.random.binomial(size=input.shape, n=1, p=0.5)
    c = (a==0) * b
    return input * a + c


#data preparation
flattened_x_train = x_train.reshape(-1,784)
flattened_x_train_seasoned = salt_and_pepper(flattened_x_train, noise_level=0.4)

flattened_x_test = x_test.reshape(-1,784)
flattened_x_test_seasoneed = salt_and_pepper(flattened_x_test, noise_level=0.4)


In [13]:

latent_dim = 96  

input_image = keras.Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_image)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_image, decoded)
encoder_only = keras.Model(input_image, encoded)

encoded_input = keras.Input(shape=(latent_dim,))
decoder_layer = Sequential(autoencoder.layers[-2:])
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [14]:
fit_info_AE = autoencoder.fit(
    flattened_x_train_seasoned,
    flattened_x_train,
    epochs=32,
    batch_size=64,
    shuffle=True,
    validation_data=(flattened_x_test_seasoneed, flattened_x_test),
)


Epoch 1/32
938/938 [==============================] - 11s 11ms/step - loss: 0.2423 - val_loss: 0.1520
Epoch 2/32
938/938 [==============================] - 9s 10ms/step - loss: 0.1489 - val_loss: 0.1381
Epoch 3/32
938/938 [==============================] - 10s 11ms/step - loss: 0.1373 - val_loss: 0.1338
Epoch 4/32
938/938 [==============================] - 10s 10ms/step - loss: 0.1320 - val_loss: 0.1303
Epoch 5/32
938/938 [==============================] - 9s 9ms/step - loss: 0.1283 - val_loss: 0.1275
Epoch 6/32
938/938 [==============================] - 10s 10ms/step - loss: 0.1262 - val_loss: 0.1262
Epoch 7/32
938/938 [==============================] - 10s 11ms/step - loss: 0.1243 - val_loss: 0.1258
Epoch 8/32
938/938 [==============================] - 9s 10ms/step - loss: 0.1230 - val_loss: 0.1242
Epoch 9/32
938/938 [==============================] - 10s 11ms/step - loss: 0.1220 - val_loss: 0.1237
Epoch 10/32
938/938 [==============================] - 9s 10ms/step - loss: 0.1211 - v

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2206d3be-0005-47e9-b8a0-2566096b1bac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>